In [97]:
import SimFunctions
import SimClasses
import SimRNG
import simpy
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import probplot, kstest
import pickle
from scipy.stats.distributions import norm, expon, erlang, lognorm, weibull_min, gamma
warnings.filterwarnings("ignore")
# fix random number seed
np.random.seed(1)


In [98]:
ZSimRNG = SimRNG.InitializeRNSeed()
Calendar = SimClasses.EventCalendar()
TheQueues = []
TheCTStats = []
TheDTStats = []
TheResources = []


# Classes

In [99]:
class Station:
    def __init__(self, station_id, level, capacity):
        self.id = station_id
        self.level = level
        self.capacity = capacity
        self.bikes = {}
        self.bike_list = []
        for i in range(level):
            bike_id = f"{station_id}-{i+1}"  # create unique bike ID
            self.bikes[bike_id] = True  # mark bike as available
            self.bike_list.append(bike_id)
            
    def rent_bike(self):
        # Request a bike from the station
        if self.level > 0:
            if self.bike_list:
                random_index = np.random.randint(0, len(self.bike_list))
                bike_id = self.bike_list.pop(random_index)
                self.level -= 1
                return bike_id
        return None
    
    def return_bike(self, bike_id):
        # # Return a bike to the station
        self.bike_list.append(bike_id)
        self.level += 1

    def Get_Bike_List(self):
        return self.bike_list
    
class Customer:
    def __init__(self, customer_id, start_station, bike = None):
        self.customer_id = customer_id
        self.start_station = start_station
        self.end_id = 0
        self.station_level = 0
        self.bike = bike

    def rent_bike(self):
        station = StationDict[self.start_station]
        self.station_level = station.level - 1
        self.bike = station.rent_bike()
        print("(CUSTOMER ARRIVAL) Customer ID: {} | Bike ID {} | Station ID: {} | level {}".format(self.customer_id, self.bike, self.start_station, station.level))
        SimFunctions.Schedule(Calendar, "Departure", 2*SimRNG.Lognormal(0.1, np.sqrt(0.066), 6))
        
    def return_bike(self, end_station, bike):
        destination_station = StationDict[end_station]
        self.station_level = destination_station.level
        destination_station.return_bike(bike)
        

# Functions

In [100]:
def NextCustomerID():
    if not hasattr(NextCustomerID, "counter"):
        NextCustomerID.counter = 0
    NextCustomerID.counter += 1
    return NextCustomerID.counter

def Customer_Arrival_Rate(station_id):
    U = SimRNG.Uniform(0, 1, 5)
    if U < (1/3):
        station_id = 1
    elif U < (2/3) and U > (1/3):
        station_id = 2
    else:
        station_id = 3
    return station_id

def Customer_Arrival(station_id, empty_error, CustomerList):
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(0.5, 1))
    station = StationDict[station_id]
    customer_id = NextCustomerID()
    customer = Customer(customer_id, station_id)
    CustomerList.append(customer)
    customer.start_station = station_id

    # Station is empty
    if station.level <= 0:
        print(f"    EMPTY -- Customer Arrives at STATION {station.id} -- EMPTY")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        empty_error += 1
    else:
        customer.rent_bike()

    return Customer_Arrival_Rate(station_id), empty_error, customer

def Bike_Arrival(end_id, Full_Error, CustomerList):
    last_customer = CustomerList[-1]
    last_customer.end_id = end_id
    destination_station = StationDict[end_id] 
    #bike = destination_station
    for customer in CustomerList:
        if customer.end_id == end_id and customer.bike is not None:
            print("(BIKE RETURNED) Customer ID: {} | Bike ID: {} | From ST: {} | To ST: {} -> Level: {}".format(customer.customer_id, customer.bike, customer.start_station, customer.end_id, destination_station.level))
            print("-------------------------------------------------------------------------")
            #destination_station.return_bike(customer.bike)
            customer.return_bike(end_id, customer.bike)
            CustomerList.remove(customer)
            return Full_Error
    #print(f"    BIKE NOT FOUND -- Bike Arrives at STATION {destination_station.id}")
    if destination_station.level >= destination_station.capacity:
        print(f"    FULL -- STATION {destination_station.id} -- FULL")
        print("Full Error", Full_Error)

        Full_Error = Full_Error + 1
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(2, 5))
        return Full_Error
    
    return Full_Error



def Retrial(end_id, Full_Error, CustomerList):
    print("Retrial")
        
    last_customer = CustomerList[-1]
    last_customer.end_id = end_id
    destination_station = StationDict[end_id]

    if destination_station.level < destination_station.capacity:
        for customer in CustomerList:
            if customer.end_id == end_id and customer.bike is not None:
                print("     Bike has succesfully retried and returned bike to Station", destination_station.id)
                print("(BIKE RETURNED) Customer ID: {} | Bike ID: {} | From ST: {} | To ST: {} -> Level: {}".format(customer.customer_id, customer.bike, customer.start_station, customer.end_id, destination_station.level))
                print("-------------------------------------------------------------------------")
                destination_station.return_bike(customer.bike)
                #customer.return_bike(end_id, customer.bike)
                CustomerList.remove(customer)
                return Full_Error
    else:
        # Station full or bike not found, schedule a retrial event
        print(f"    FULL -- STATION {destination_station.id} -- FULL")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(2, 5))
        print("Full Error", Full_Error)
        Full_Error = Full_Error + 1
        return Full_Error
    return Full_Error


def Departure(station_id, customer):
    station = StationDict[station_id]
    print("(DEPARTING) Station ID: {} | Customer ID: {} | Level {} | ".format(customer.start_station, customer.customer_id, customer.station_level))

    #print("(DEPARTING) Station ID: {} | Customer ID: {} | Level {} | ".format(station.id, customer.customer_id, station.level))
    print("-------------------------------------------------------------------------")
    end_id = Destination(station)
    customer.end_id = end_id
    SimFunctions.Schedule(Calendar, "Bike_Arrival", 15 *SimRNG.Lognormal(0.1, np.sqrt(0.066), 4))
    return end_id


def Destination(station):
    #station = StationDict[station_id]
    station_id = station.id
    U = SimRNG.Uniform(0, 1, 3)
    if U < (1/3):
        end_id = 1
    elif U < (2/3) and U > (1/3):
        end_id = 2
    else:
        end_id = 3
    #end_id = 2 if U > 1/2 else 1

    end_station = StationDict[end_id]
    print("(DESTINATION) Station ID: {} | Level: {}".format(end_station.id, end_station.level))
    #print("-------------------------------------------------------------------------")
    return end_id


# Simulation

In [105]:
ZSimRNG = SimRNG.InitializeRNSeed()

Calendar = SimClasses.EventCalendar()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

CustomerList = []

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5
end_id = 1
Full_Error = 0
Empty_Error = 0

for reps in range(0, 1, 1):
    
    inital_station_id = np.random.random_integers(1, 3)
    station_id = inital_station_id
    
    # Initialize the stations and create a dictionary mapping station IDs to Station instances
    Stations = [Station(station_id=1, level=5, capacity=5), Station(station_id=2, level=35, capacity=40), Station(station_id=3, level=25, capacity=25)]
    StationDict = {station.id: station for station in Stations}

    SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(MeanTBA, 1))

    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Customer_Arrival":
        Customer_Arrival(inital_station_id, Empty_Error, CustomerList)


    for hours in [8, 8.3, 9, 9.3, 10, 10.3, 11, 11.3, 12]:  # 30 min intervals
        print(f"Clock Hour | {hours}")
        for minutes in [i for i in range(1, 31)]:  # 30 min intervals
            print("Minute | {}".format(minutes))
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            
            if NextEvent.EventType == "Customer_Arrival":
                station_id, Empty_Error, customer = Customer_Arrival(station_id, Empty_Error, CustomerList)

            elif NextEvent.EventType == "Departure":
                end_id = Departure(station_id, customer)
            
            elif NextEvent.EventType == "Bike_Arrival":
                Full_Error = Bike_Arrival(end_id, Full_Error, CustomerList)
            
            elif NextEvent.EventType == "Retrial":
                Full_Error = Retrial(end_id, Full_Error, CustomerList)
               
    print(StationDict[1].Get_Bike_List())
    print(StationDict[2].Get_Bike_List())
    print(StationDict[3].Get_Bike_List())
    print("Num of Full Errors:", Full_Error)
    print("Num of Empty Errors:", Empty_Error)

(CUSTOMER ARRIVAL) Customer ID: 367 | Bike ID 2-19 | Station ID: 2 | level 34
Clock Hour | 8
Minute | 1
(DEPARTING) Station ID: 2 | Customer ID: 366 | Level 46 | 
-------------------------------------------------------------------------
(DESTINATION) Station ID: 2 | Level: 34
Minute | 2
(BIKE RETURNED) Customer ID: 367 | Bike ID: 2-19 | From ST: 2 | To ST: 2 -> Level: 34
-------------------------------------------------------------------------
Minute | 3
(CUSTOMER ARRIVAL) Customer ID: 368 | Bike ID 2-32 | Station ID: 2 | level 34
Minute | 4
(DEPARTING) Station ID: 2 | Customer ID: 368 | Level 34 | 
-------------------------------------------------------------------------
(DESTINATION) Station ID: 2 | Level: 34
Minute | 5
(BIKE RETURNED) Customer ID: 368 | Bike ID: 2-32 | From ST: 2 | To ST: 2 -> Level: 34
-------------------------------------------------------------------------
Minute | 6
(CUSTOMER ARRIVAL) Customer ID: 369 | Bike ID 2-35 | Station ID: 2 | level 34
Minute | 7
(DEPARTI